# Prompt Tuning Example


In [1]:
pip install peft

     |████████████████████████████████| 183 kB 23.1 MB/s eta 0:00:01
     |████████████████████████████████| 8.5 MB 90.7 MB/s eta 0:00:01
     |████████████████████████████████| 330 kB 136.7 MB/s eta 0:00:01
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
     |████████████████████████████████| 1.3 MB 98.2 MB/s eta 0:00:01
     |████████████████████████████████| 755.5 MB 33 kB/s s eta 0:00:011     |████████████████████▉           | 491.7 MB 118.2 MB/s eta 0:00:03     |█████████████████████▎          | 501.8 MB 118.2 MB/s eta 0:00:03
     |████████████████████████████████| 78 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 736 kB 142.3 MB/s eta 0:00:01
     |████████████████████████████████| 279 kB 121.5 MB/s eta 0:00:01
     |████████████████████████████████| 777 kB 150.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 116.4 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 1.9 MB/s  eta 0:00:0

In [5]:
pip install datasets

  Using cached datasets-2.17.1-py3-none-any.whl.metadata (20 kB)
  Using cached pyarrow-15.0.0-cp38-cp38-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.4.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py38-none-any.whl.metadata (7.1 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached aiohttp-3.9.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached frozenlist-1.4.1-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 k

In [6]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
## import libraries
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = 'bigscience/bloomz-560m'
NUM_VIRTUAL_TOKENS = 4
NUM_EPOCHS = 5

In [8]:
## load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                          trust_remote_code = True)

tokenizer_config.json: 100%|██████████| 222/222 [00:00<00:00, 44.8kB/s]
tokenizer.json: 100%|██████████| 14.5M/14.5M [00:00<00:00, 97.4MB/s]
special_tokens_map.json: 100%|██████████| 85.0/85.0 [00:00<00:00, 80.7kB/s]
config.json: 100%|██████████| 715/715 [00:00<00:00, 601kB/s]
model.safetensors: 100%|██████████| 1.12G/1.12G [00:09<00:00, 116MB/s]


In [19]:
## create a function that returns the outputs from the model we have received above, and inputs.
def get_outputs(model, inputs, max_new_tokens = 100):
    outputs = model.generate(
        input_ids = inputs['input_ids'],
        attention_mask = inputs['attention_mask'],
        max_new_tokens = max_new_tokens,
        repetition_penalty = 1.5, 
        early_stopping = True,
        eos_token_id = tokenizer.eos_token_id,
        num_beams = 6
    )

    return outputs

In [93]:
## get a smaple response from the above get_output and existing foundational models
test_prompt = 'I want you to act as a motivational coach.'

input_prompt = tokenizer(test_prompt, return_tensors = 'pt')
foundational_outputs_prompt = get_outputs(foundational_model, input_prompt, max_new_tokens = 50)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens = True))

["I want you to act as a motivational coach. Don't be afraid to ask questions"]


In [94]:
## get a smaple response from the above get_output and existing foundational models
test_sentence = 'There are two things that matter:'

input_sentences = tokenizer(test_sentence, return_tensors = 'pt')
foundational_outputs_sentence = get_outputs(foundational_model, input_sentences, max_new_tokens = 50)

print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens = True))

['There are two things that matter: quality and quantity. quality matters more than quantity.']


In [95]:
## preparing the datasets
from datasets import load_dataset

dataset_prompt = 'fka/awesome-chatgpt-prompts'

#create the dataset to create prompts
data_prompt = load_dataset(dataset_prompt)
data_prompt = data_prompt.map(lambda samples:tokenizer(samples['prompt']), batched = True)
train_sample_prompt = data_prompt['train'].select(range(50))

train_sample_prompt = train_sample_prompt.remove_columns(['act'])

display(train_sample_prompt)

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 50
})

In [96]:
pd.DataFrame(train_sample_prompt)

,prompt,input_ids,attention_mask
0,I want you to act as a linux terminal. I will ...,"[44, 4026, 1152, 427, 1769, 661, 267, 104105, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"I want you to act as an English translator, sp...","[44, 4026, 1152, 427, 1769, 661, 660, 7165, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,I want you to act as an interviewer. I will be...,"[44, 4026, 1152, 427, 1769, 661, 660, 33322, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,I want you to act as a javascript console. I w...,"[44, 4026, 1152, 427, 1769, 661, 267, 49760, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,I want you to act as a text based excel. you'l...,"[44, 4026, 1152, 427, 1769, 661, 267, 5484, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,I want you to act as an English pronunciation ...,"[44, 4026, 1152, 427, 1769, 661, 660, 7165, 14...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,I want you to act as a spoken English teacher ...,"[44, 4026, 1152, 427, 1769, 661, 267, 93949, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,I want you to act as a travel guide. I will wr...,"[44, 4026, 1152, 427, 1769, 661, 267, 25008, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,I want you to act as a plagiarism checker. I w...,"[44, 4026, 1152, 427, 1769, 661, 267, 159667, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,I want you to act like {character} from {serie...,"[44, 4026, 1152, 427, 1769, 3269, 731, 84491, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [97]:
## prepare a second dataset 
dataset_sentences = load_dataset('Abirate/english_quotes')

data_sentences = dataset_sentences.map(lambda samples:tokenizer(samples['quote']), batched = True)
train_sample_sentences = data_sentences['train'].select(range(25))
train_sample_sentences = train_sample_sentences.remove_columns(['author','tags'])
display(train_sample_sentences)

Dataset({
    features: ['quote', 'input_ids', 'attention_mask'],
    num_rows: 25
})

In [98]:
pd.DataFrame(train_sample_sentences)

,quote,input_ids,attention_mask
0,“Be yourself; everyone else is already taken.”,"[1502, 17143, 33218, 30, 39839, 4384, 632, 112...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,"“I'm selfish, impatient and a little insecure....","[1502, 10203, 239002, 15, 136192, 1049, 530, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,“Two things are infinite: the universe and hum...,"[1502, 35417, 11217, 1306, 61759, 29, 368, 713...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"“So many books, so little time.”","[1502, 6895, 7112, 38695, 15, 1427, 10512, 350...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,“A room without books is like a body without a...,"[119533, 22630, 7160, 38695, 632, 3269, 267, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,"“Be who you are and say what you feel, because...","[1502, 17143, 5268, 1152, 1306, 530, 5894, 359...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,“You've gotta dance like there's nobody watchi...,"[1502, 124002, 83213, 59020, 3269, 20242, 8839...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,“You know you're in love when you can't fall a...,"[1502, 5448, 4472, 11700, 361, 19134, 3262, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,"“You only live once, but if you do it right, o...","[1502, 5448, 3804, 20152, 14275, 15, 1965, 132...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,“Be the change that you wish to see in the wor...,"[1502, 17143, 368, 7458, 861, 1152, 26338, 427...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [99]:
## time to fine tune with Parameter Efficient Fine Tuning
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

generator_config = PromptTuningConfig(
    task_type = TaskType.CAUSAL_LM, # this makes the model generate text
    prompt_tuning_init = PromptTuningInit.RANDOM, # initialise the virtual tokens with random numbers
    num_virtual_tokens = NUM_VIRTUAL_TOKENS, # number of virtual tokens to add and train
    tokenizer_name_or_path = model_name
)

In [100]:
peft_model_prompt = get_peft_model(foundational_model, generator_config)
print(peft_model_prompt.print_trainable_parameters())

trainable params: 4,096 || all params: 559,218,688 || trainable%: 0.0007324504863471229
None


In [101]:
peft_model_sentences = get_peft_model(foundational_model, generator_config)
print(peft_model_sentences.print_trainable_parameters())

trainable params: 4,096 || all params: 559,218,688 || trainable%: 0.0007324504863471229
None


In [102]:
## start to create the training configuration
from transformers import TrainingArguments

def create_training_arguments(path, learning_rate = 0.0035, epochs = 6):
    training_args = TrainingArguments(
        output_dir = path,
        auto_find_batch_size = True,
        learning_rate = learning_rate,
        num_train_epochs = epochs
    )

    return training_args

In [103]:
import os

In [104]:
## create directories to hold the model when they don't exist

working_dir = './peft'

## it is recommended to store the models seperately
output_dir_prompt = os.path.join(working_dir, 'peft_outputs_prompt')
output_dir_sentences = os.path.join(working_dir, 'peft_outputs_sentences')

## create the directories if they don't exist
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
if not os.path.exists(output_dir_prompt):
    os.mkdir(output_dir_prompt)
if not os.path.exists(output_dir_sentences):
    os.mkdir(output_dir_sentences)


In [105]:
training_args_prompt = create_training_arguments(output_dir_prompt, 0.003, NUM_EPOCHS)
training_args_sentences = create_training_arguments(output_dir_sentences, 0.0035, NUM_EPOCHS)

In [106]:
from transformers import Trainer, DataCollatorForLanguageModeling 

def create_trainer(model, training_args, train_dataset):
    trainer = Trainer(
        model=model,
        args = training_args,
        train_dataset = train_dataset,
        data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
    )

    return trainer

In [107]:
## time to finetune the models
trainer_prompt = create_trainer(peft_model_prompt, training_args_prompt, train_sample_prompt)
trainer_prompt.train()

trainer_sentences = create_trainer(peft_model_sentences, training_args_sentences, train_sample_sentences)
trainer_sentences.train()

Step,Training Loss


Step,Training Loss


TrainOutput(global_step=5, training_loss=4.086028289794922, metrics={'train_runtime': 50.9607, 'train_samples_per_second': 2.453, 'train_steps_per_second': 0.098, 'total_flos': 28795358208000.0, 'train_loss': 4.086028289794922, 'epoch': 5.0})

In [108]:
## save the models
trainer_prompt.model.save_pretrained(output_dir_prompt)
trainer_sentences.model.save_pretrained(output_dir_sentences)

In [109]:
from peft import PeftModel

loaded_model_prompt = PeftModel.from_pretrained(foundational_model,
                                         output_dir_prompt,
                                         #device_map='auto',
                                         is_trainable=False)

In [110]:
## inference
loaded_model_prompt_outputs = get_outputs(loaded_model_prompt, input_prompt)
print(tokenizer.batch_decode(loaded_model_prompt_outputs, skip_special_tokens = True))


['I want you to act as a motivational coach. I want you to help me make my life better']


In [111]:
loaded_model_sentences = PeftModel.from_pretrained(foundational_model,
                                         output_dir_sentences,
                                         #device_map='auto',
                                         is_trainable=False)

In [113]:
## inference
loaded_model_sentences_outputs = get_outputs(loaded_model_sentences, input_sentences)
print(tokenizer.batch_decode(loaded_model_sentences_outputs, skip_special_tokens = True))

['There are two things that matter: quality and quantity. quality matters more than quantity']
